# What is NumPy

Numpy is Python library for scientific computing. It’s main object is the homogeneous multidimensional array. It is a table of elements, all of the same type, indexed by a tuple of positive integers. 

## Why numpy arrays can accelerate your applications?

A NumPy ndarray array is described by metadata: number of dimensions, shape, data type, and so on, and the actual data stored in it. The data is stored in a homogeneous and contiguous blocks of memory, at a particular address in system memory (Random Access Memory, or RAM). This is the main difference with a pure Python data structures, in which the items are scattered across the system memory. This aspect is the critical feature that makes NumPy arrays efficient.

## Why is this so important? 

Array computations can be implemented efficiently in a low-level language like C (and a large part of NumPy is actually written in C). Knowing the address of the memory block and the data type, it is just simple arithmetic to loop over all items, for example. There would be a significant overhead to do that in Python with a list.

Spatial locality in memory access patterns results in significant performance gains, notably thanks to the CPU cache. 

Data elements are stored contiguously in memory, so that NumPy can take advantage of vectorized instructions on modern CPUs, like Intel's SSE and AVX, AMD's XOP, and so on. 

NumPy can be linked to highly optimized linear algebra libraries like BLAS and LAPACK, for example through the Intel Math Kernel Library (MKL). A few specific matrix computations may also be multithreaded, taking advantage of the power of modern multicore processors.

In conclusion, storing data in a contiguous blocks of memory ensures that the architecture of modern CPUs is used optimally, in terms of memory access patterns, CPU cache, and vectorized instructions.

# NumPy performance improvements

With NumPy arrays, you can achieve significant performance speedups over native Python, particularly when your computations follow the Single Instruction, Multiple Data (SIMD) paradigm. 

In the following we are going to discuss some tricks to help improve your Python code when using NumPy arrays.

## NumPy universal functions

NumPy provides familiar mathematical functions such as `sin`, `cos`, and `exp`. In NumPy, these are called “universal functions”(ufunc). Within NumPy, these functions operate elementwise on an array, producing an array as output.

In [1]:
import numpy as np
B = np.arange(3)
B

array([0, 1, 2])

In [2]:
np.exp(B)

array([ 1.        ,  2.71828183,  7.3890561 ])

In [3]:
np.sqrt(B)

array([ 0.        ,  1.        ,  1.41421356])

In [4]:
C = np.array([2., -1., 4.])
np.add(B, C)

array([ 2.,  0.,  6.])

Other NumPy ufuncs include:

all, any, apply_along_axis, argmax, argmin, argsort, average, bincount, ceil, clip, conj, corrcoef, cov, cross, cumprod, cumsum, diff, dot, floor, inner, inv, lexsort, max, maximum, mean, median, min, minimum, nonzero, outer, prod, re, round, sort, std, sum, trace, transpose, var, vdot, vectorize, where

## Simple example: dot product

Let us start with our running example of dot product computation of two vectors. Simply substituting Python data structures (lists) with NumPy arrays and using NumPy's universal functions can significantly speedup the calculation

In [5]:
def dot_product(vec1,vec2):
    result = 0.0
    n = len(vec1)
    for i in range(n):
        result += vec1[i]*vec2[i]
    return result

vec1 = [x for x in range(1000000)]
vec2 = [x for x in range(1000000)]
%time dot_product(vec1,vec2)

CPU times: user 154 ms, sys: 9 ms, total: 163 ms
Wall time: 162 ms


3.3333283333312755e+17

Simply re-writing the same code using numpy arrays we get a significant improvement:

In [6]:
import numpy as np

vec1 = np.arange(1000000)
vec2 = np.arange(1000000)
%time np.dot(vec1,vec2)

CPU times: user 1 ms, sys: 0 ns, total: 1 ms
Wall time: 1.35 ms


333332833333500000

## Avoid unnecessary array copies

Avoid unnecessary array copies is essential in order to save time and memory. In that respect, we will need to dig into the internals of NumPy.

The first step when looking for unnecessary array copies is to find out the location of arrays in memory. The `id()` method does just that:

In [7]:
a = np.zeros(10)
aid = id(a)
aid

139851908514560

## In-place operations vs implicit-copy

Array computations can involve `in-place operations` (first example below: the array is modified) or `implicit-copy operations` (second example: a new array is created).

In [8]:
#in-place
a *= 2
id(a) == aid

True

In [9]:
#implicit copy
c = a * 2
id(c) == aid

False

Be sure to choose the type of operation you actually need, but keep in mind that implicit-copy operations are significantly slower, as shown here:

In [10]:
%%timeit a = np.zeros(10000000)
a *= 2

100 loops, best of 3: 8.73 ms per loop


In [11]:
%%timeit a = np.zeros(10000000)
b = a * 2

10 loops, best of 3: 30.2 ms per loop


#### Reshaping an array 

Reshaping may or may not involve a copy. For instance, reshaping a 2D matrix does not involve a copy, unless it is transposed:

In [12]:
a = np.zeros((10, 10))
a

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [13]:
aid = id(a)
aid

139851908515120

Reshaping an array while preserving its order does not trigger a copy.

In [14]:
b = a.reshape((1, -1))
b

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [15]:
print a.shape
print b.shape

(10, 10)
(1, 100)


In [16]:
#Does it print out True or False for you? For me it depends on the system... 
id(b) == aid

False

Transposing an array changes its order so that a reshape triggers a copy.

In [18]:
c = a.T.reshape((1, -1))
id(c) == aid

False

Therefore, reshaping with transpose will be slower.

The `flatten` and `ravel` methods of an array reshape it into a 1D vector (flattened array). The former method always returns a copy, whereas the latter returns a copy only if necessary (so it's significantly faster too, especially with large arrays):

In [19]:
d = a.flatten()
id(d) == aid

False

In [20]:
d.shape

(100,)

In [21]:
e = a.ravel()
id(e) == aid

False

In [22]:
e.shape

(100,)

In [23]:
%timeit a.flatten()

The slowest run took 11.91 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 841 ns per loop


In [24]:
%timeit a.ravel()

The slowest run took 16.17 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 251 ns per loop


## Broadcasting rules

Broadcasting rules allow you to make computations on arrays with different but compatible shapes. In other words, you don't always need to reshape or tile your arrays to make their shapes match. 

An example of broadcasting in practice:

In [25]:
x = np.arange(4)
xx = x.reshape(4,1)
y = np.ones(5)
z = np.ones((3,4))

x.shape

(4,)

In [26]:
y.shape

(5,)

In [27]:
x + y

ValueError: operands could not be broadcast together with shapes (4,) (5,) 

In [28]:
xx.shape

(4, 1)

In [29]:
y.shape

(5,)

In [30]:
(xx + y).shape

(4, 5)

In [31]:
xx + y

array([[ 1.,  1.,  1.,  1.,  1.],
       [ 2.,  2.,  2.,  2.,  2.],
       [ 3.,  3.,  3.,  3.,  3.],
       [ 4.,  4.,  4.,  4.,  4.]])

In [32]:
x.shape

(4,)

In [33]:
z.shape

(3, 4)

In [34]:
(x + z).shape

(3, 4)

In [35]:
x + z

array([[ 1.,  2.,  3.,  4.],
       [ 1.,  2.,  3.,  4.],
       [ 1.,  2.,  3.,  4.]])

### Tiling

Tile operation simply repeats the numbers of elements in an array. Given an array [1,2,3], then np.tile([1,2,3], 2) will repeat the elements twice and make a new array:

In [36]:
ar = [1]

In [37]:
np.tile(ar, 2)

array([1, 1])

In [38]:
np.tile(ar, 3)

array([1, 1, 1])

In [39]:
new_ar = [1,2,3]

In [40]:
np.tile(new_ar, 2)

array([1, 2, 3, 1, 2, 3])

### Tiling vs broadcasting

The following example illustrates two ways of doing an outer product between two vectors: the first method involves array tiling, the second one involves broadcasting:

In [41]:
n = 1000
a = np.arange(n)
ac = a[:, np.newaxis]
ar = a[np.newaxis, :]
%timeit np.tile(ac, (1, n)) * np.tile(ar, (n, 1))

100 loops, best of 3: 7.56 ms per loop


In [42]:
p = np.tile(ac, (1, n)) * np.tile(ar, (n, 1))
p.shape

(1000, 1000)

In [43]:
%timeit ar * ac

100 loops, best of 3: 2.18 ms per loop


In [44]:
p = ar * ac
p.shape

(1000, 1000)

## Making efficient selections in arrays with Numpy

NumPy offers multiple ways of selecting slices of arrays. Array views refer to the original data buffer of an array, but with different offsets, shapes and strides. They only permit strided selections (i.e. with linearly spaced indices). NumPy also offers specific functions to make arbitrary selections along one axis. Finally, fancy indexing is the most general selection method, but it is also the slowest as we will see in this recipe. Faster alternatives should be chosen when possible.

Let's create an array with a large number of rows. We will select slices of this array along the first dimension.

In [45]:
n, d = 100000, 100
a = np.random.random_sample((n, d))
aid = id(a)

In [46]:
a.shape

(100000, 100)

### Array views and fancy indexing

Let's select one in every ten rows, using two different methods (array view and fancy indexing).

In [47]:
#array view
b1 = a[::10]
#fancy indexing with array
b2 = a[np.arange(0, n, 10)]
np.arange(0, n, 10)

array([    0,    10,    20, ..., 99970, 99980, 99990])

In [48]:
np.array_equal(b1, b2)

True

The view refers to the original data buffer, whereas fancy indexing yields a copy.

In [49]:
id(b1) == aid, id(b2) == aid

(False, False)

Let's compare the performance of both methods.

In [50]:
%timeit a[::10]

The slowest run took 51.08 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 233 ns per loop


In [51]:
%timeit a[np.arange(0, n, 10)]

1000 loops, best of 3: 1.44 ms per loop


Fancy indexing is several orders of magnitude slower as it involves copying a large array.

### Alternatives to fancy indexing: list of indices

When non-strided selections need to be done along one dimension, array views are not an option. However, alternatives to fancy indexing still exist in this case. Given a list of indices, NumPy's function take performs a selection along one axis.

In [56]:
#fancy indexing
i = np.arange(0, n, 10)
b1 = a[i]
#numpy take selection
b2 = np.take(a, i, axis=0)
np.array_equal(b1, b2)

True

In [57]:
%timeit a[i]

100 loops, best of 3: 2.87 ms per loop


In [58]:
%timeit np.take(a, i, axis=0)

100 loops, best of 3: 3.31 ms per loop


In this case I did not observe a performance difference, but I thought it is worth mentioning.

Fancy indexing is the most general way of making completely arbitrary selections of an array. However, more specific and faster methods often exist and should be preferred when possible.

Array views should be used whenever strided selections have to be done, but one needs to be careful about the fact that views refer to the original data buffer.